# Policy Analysis Notebook

The purpose of this notebook is to facilitate the generation of shared micro-mobility policy analysis.
It serves as the central hub from which parameters are declared and subsequent functions for data processing, analysis, and visualisation are run from, while keeping the code and "back-end" processes hidden and concise.

### Imports

In [ ]:
import datetime as dt
import scripts.style
from scripts.api_smhi_weather import *
from scripts.data_processing import *
from scripts.data_visualisations import *

## Policy Parameters

**Path**: path to where the trip data is located, ideally placed in /data, and assumed to be a *CSV file*

**City Name**: the place where the policy was implemented

**Before Start / End date**: The timeframe of the selected period before the policy introduction (earliest and latest)

**After Start / End date**: The timeframe of the selected period after the policy introduction (earliest and latest)

**City Coordinates**: The area to which to restrict the policy analysis to, in the following order: North, South, East, West.
Used for data cleaning (i.e. set to broad perimeter if data is already cleaned of spatial outliers), e.g. from previous datasets:
* STH - 59.39, 59.28, 18.18, 17.94;
* GBG - 57.74, 57.67, 12.02, 11.90

##### <mark>Update the parameters below</mark>

In [ ]:
data_path = 'data/data_raw.csv'
city_name = 'Stockholm'

before_start_date = dt.date(2022, 8, 5)
before_end_date = dt.date(2022, 8, 27)

after_start_date = dt.date(2022, 9, 15)
after_end_date = dt.date(2022, 9, 30)

city_coordinates = (59.39, 59.28, 18.18, 17.94)

## Preprocessing

The data is read and parsed according to the methods in `scripts/data_processing`. A brief visualisation of the distribution over days, weekdays and operators follows as a sanity check.

In [ ]:
df = pd.read_csv(data_path)
df = parse_data(df)
df = clean_data(df, city_coordinates)

In [ ]:
vis_dataset_overview(df, before_start_date, after_end_date)

In [ ]:
# counts number of hours per day where no data was recorded, visualisation follows later
df_missing = missing_data_hours(df, before_start_date, before_end_date, after_start_date, after_end_date)

### Weather Parameters

In order to filter out any potential effects of the weather on usage behaviour, such as intense rain-/snowfall or very low temperatures, weather data from [SMHI](https://www.smhi.se/data/meteorologi/ladda-ner-meteorologiska-observationer/#param=airTemperatureMinAndMaxOnceEveryDay,stations=active) is fetched.

Currently, the parameters are minimum and maximum temperature, as well as daily precipitation in mm. More information can be found on SMHI's website. The cells below aid in selecting a weather station that records all three parameters, and is within a radius of 30km of the dataset's geographic midpoint.e

In [ ]:
# Min temperature, max temperature, and precipitation in mm (daily sum)
params = [19, 20, 5]
data_mid_lat = df.o_lat.mean()
data_mid_lon = df.o_lng.mean()

weather_stations = get_available_stations(params, data_mid_lat, data_mid_lon)
map_stations(weather_stations, data_mid_lat, data_mid_lon)

##### <mark>Select a weather station from the plot above and declare its ID (hover to retrieve) in the cell below:</mark>

In [ ]:
station_key = 98230 #change ID to desired station
df_weather = fetch_weather_data(station_key, params, before_start_date, after_end_date)

In [ ]:
plot_weather(df_weather, df_missing, city_name)

### Data Selection

The above plot shows the weather conditions during the timeframe as well as number of hours during which no data/trips have been recorded (e.g. due to system outage), such that such outlier days can be filtered out. Additionally, the dataset is split into before and after.

##### <mark>Select in the cell below which days should not be included in the dataset, based on weather conditions and missing data throughout the day:</mark>

In [ ]:
# dataset is split into before and after
df_before = df[(df.isodate >= pd.to_datetime(before_start_date)) & (df.isodate <= pd.to_datetime(before_end_date))]
df_after = df[(df.isodate >= pd.to_datetime(after_start_date)) & (df.isodate <= pd.to_datetime(after_end_date))]

In [ ]:
# update days which should be filtered out
df_before = df_before[~df_before.Date.isin([5, 12, 14, 20, 25, 28])]
df_after = df_after[~df_after.Date.isin([16, 17, 18, 19, 20, 21, 22, 26, 30])]

### Data Preparation

Given the cleaned and selected data, further calculations are performed on the data. For each trip, the likelihood of each alternative mode of transport is computed, the data is partitioned into types of weekdays, and an aggregated grid is created.

In [ ]:
for frame in [df_before, df_after]:
    frame = calculate_mode_likelihood_ghg(frame)
    frame = frame.dropna(how='any')

In [ ]:
weekends = ['Friday','Saturday', 'Sunday']
weekdays_before = df_before[~df_before['day'].isin(weekends)]
weekdays_after = df_after[~df_after['day'].isin(weekends)]

fri_before = df_before[df_before['day'].isin(['Friday'])]
fri_after = df_after[df_after['day'].isin(['Friday'])]

weekends = ['Saturday', 'Sunday']
weekends_before = df_before[df_before['day'].isin(weekends)]
weekends_after = df_after[df_after['day'].isin(weekends)]

<mark> Select the grid shape ('square', 'hexagon', or 'shapefile').</mark><br>
**Note:** If using shapefile, make sure the necessary files are in the ```/data``` directory, or you update the path accordingly in the code.


<mark> Select the grid resolution, if using 'square' or 'hexagon'.</mark> Cell statistics/resolutions for hexagons can be found *[here](https://h3geo.org/docs/core-library/restable/#edge-lengths)*.

In [ ]:
grid_shape = 'square' # 'shape', 'hexagon', 'shapefile'
grid_size = 0.003 # for SQUARE: edge length in km [e.g.: 0.002], for HEXAGON: resolution (int) [e.g.: 9]
min_data = 30 # minimum data points per day for a zone to be considered

In [ ]:
gdf_before = create_zones(df_before, grid_shape, grid_size, city_coordinates, min_data, 'o') # origin-based
gdf_before_d =  create_zones(df_before, grid_shape, grid_size, city_coordinates, min_data, 'd') # destination-based

gdf_after =  create_zones(df_after, grid_shape, grid_size, city_coordinates, min_data, 'o')
gdf_after_d = create_zones(df_after, grid_shape, grid_size, city_coordinates, min_data, 'd')

## Policy Analysis

### Demand

In [ ]:
### Overall Statistics
total_size = df_before.id.size + df_after.id.size
print('Total number of trips - before: ' + str(df_before.id.size) + ' (' + str(round((df_before.id.size/total_size)*100,2)) + '%)')
print('Total number of trips - after: ' + str(df_after.id.size) + ' (' + str(round((df_after.id.size/total_size)*100,2)) + '%)\n')

# aggregate_and_describe(df_before, df_after, column_to_group_by, column_to_aggregate_on, aggregation function, table title
tripsperday = aggregate_and_describe(df_before, df_after, 'isodate', 'id', lambda x: x.count(), "Trips per Day")
tripsperhour = aggregate_and_describe(df_before, df_after, ['isodate', 'hour'], 'id', lambda x: x.count(), "Trips per Hour")
display_html(tripsperday._repr_html_() + tripsperhour._repr_html_(), raw=True)

### Histograms/PDFs
# change first parameter (density: bool) to toggle between histogram and PDF
plot_demand_pdf_throughout_weekdays(False, weekdays_before, weekends_before, fri_before, weekdays_after, weekends_after, fri_after)

#### Demand Zone Aggregation, based on Origin

In [ ]:
dims = ['demand_total','demand_dailysumavg']

a,b = plot_before_after_az(dims, [gdf_before, gdf_after], 'trips/zone')
plt.show()

ttest_traveltime = paired_ttest_az(dims, [gdf_before, gdf_after])
display(ttest_traveltime)

a, b = plot_differences_az(dims, [gdf_before, gdf_after], 'trips/zone')

##### Demand Zone Aggregation, Based on Destination

In [ ]:
dims = ['demand_total','demand_dailysumavg']

a,b = plot_before_after_az(dims, [gdf_before_d, gdf_after_d], 'trips/zone')
plt.show()

ttest_traveltime = paired_ttest_az(dims, [gdf_before_d, gdf_after_d])
display(ttest_traveltime)

a, b = plot_differences_az(dims, [gdf_before_d, gdf_after_d], 'trips/zone')

### Number of Scooters and Usage Efficiency

In [ ]:
total_scoot = pd.concat([df_before.id, df_after.id]).nunique()
print('Total distinct scooters in timeframe: ' + str(total_scoot))
print('Total number of scooters - before: ' + str(df_before.id.nunique()) + ' (' + str(round((df_before.id.nunique()/total_scoot)*100,2)) + '%)')
print('Total number of scooters - after: ' + str(df_after.id.nunique()) + ' (' + str(round((df_after.id.nunique()/total_scoot)*100,2)) + '%)\n')

scootperday = aggregate_and_describe(df_before, df_after, ['isodate'], 'id', lambda x: x.nunique(), "Scooters per Day", True, False)
scootperhour = aggregate_and_describe(df_before, df_after, ['isodate', 'hour'], 'id', lambda x: x.nunique(), "Scooters per Hour", True, False)

usetotal = aggregate_and_describe(df_before, df_after, ['id'], 'id', lambda x: x.count(), "Total Usage Efficiency", True, False)
useperday = aggregate_and_describe(df_before, df_after, ['isodate'], 'id', lambda x: x.count() / x.nunique(), "Daily Usage Efficiency", True, False)
useperhour = aggregate_and_describe(df_before, df_after, ['isodate', 'hour'], 'id', lambda x: x.count() / x.nunique(), "Hourly Usage Efficiency", True, False)

display_html(scootperday._repr_html_() + scootperhour._repr_html_() + usetotal._repr_html_() + useperday._repr_html_() + useperhour._repr_html_(), raw=True)

In [ ]:
# Plot usage efficiency by combined or distinguished operator
plot_scooter_efficiency_histogram(df_before, df_after, [['tier'], ['voi']])
plot_scooter_efficiency_histogram(df_before, df_after, [['tier','voi']])

In [ ]:
dims=['utilisationrate_avg' , 'nunique_id_dailysumavg']

a,b = plot_before_after_az(dims, [gdf_before, gdf_after], 'trips/zone')
plt.show()

ttest_traveltime = paired_ttest_az(dims, [gdf_before, gdf_after])
display(ttest_traveltime)

a, b = plot_differences_az(dims, [gdf_before, gdf_after], 'trips/zone')
plt.show()